# Determine optimal lag based on VAR
#### total range: 1988-08-31 to 2019-12-31
#### analysis range: 1990-01-31 to 2019-07-31
#### training: 2017-01-01 to 2017-12-31
#### predict: 2018-01-01 to 2018-12-31
## CS-Aaa-3MO CS-Aa-3MO CS-A-3MO CS-Baa-3MO CS-Aaa-1YR CS-Aa-1YR CS-A-1YR CS-Baa-1YR CS-Aaa-5YR CS-Aa-5YR CS-A-5YR CS-Baa-5YR
## TB-3MO-TY TB-1YR-TY TB-5YR-TY

In [1]:
start_date = '2009-01-31' #1990-01-31' # '2009-01-31'
import nb_credit_spread as cslibrary
cslib = cslibrary.creditspread()

In [24]:

ytw = cslib.get_ytw_from_date(start='1990-01-31', srcfile='YTW-All-Values.xlsx')
print(ytw.shape)

ytw = cslib.get_ytw_from_date(start=start_date, srcfile='YTW-All-Values.xlsx')
print(ytw.shape)
period = '3MO' # 3MO, 1YR, 5YR
corp = 'Aaa'
cols = [f'CS-{corp}-{period}' ,f'TB-{period}-TY']
cols = [f'CS-{corp}-{period}']

print(cols)
endog = ytw[cols]

(355, 28)
(127, 28)
['CS-Aaa-3MO']


In [26]:
def lag_VAR_selection(columns):
    '''
    get optimal lag using VAR; with 2 column vector
    '''
    from statsmodels.tsa.api import VAR
    import numpy as np

    endog = columns

    num = 30
    bic = np.zeros(shape=(num, 1))
    aic = np.zeros(shape=(num, 1))
    hqic = np.zeros(shape=(num, 1))

    for i in range(num):
        # model_fit = VAR(endog=endog).fit(maxlags=i + 1, verbose=True)
        model_fit = VAR(endog=endog).fit(maxlags=i + 1, trend='ct')
        # AIC, BIC, HQIC
        # print(f'order: {order} bic: {model_fit.bic}')
        bic[i] = model_fit.bic  # update poistion with info criteria result
        aic[i] = model_fit.aic
        hqic[i] = model_fit.hqic

    bic_min, aic_min, hqic_mid = np.min(bic), np.min(aic), np.min(hqic)
    bic_model_min, aic_model_min, hqic_model_min = np.argmin(bic), np.argmin(aic), np.argmin(hqic)

    # print('relative likelihood')
    #print(np.exp((info_crit_min - info_crit) / 2))
    #print('parameters in minimum model %s' % (model_min + 1))
    return aic_model_min + 1, bic_model_min + 1, hqic_model_min + 1

In [28]:
ytw = cslib.get_ytw_from_date(start=start_date)
print(ytw.shape)
period = '3MO' # 3MO, 1YR, 5YR
corp = 'Baa'
column = f'CS-{corp}-{period}'
cols = [column] #  ,f'TB-{period}-TY'] # only one column for ARIMA
# print(f'column: {column} lag: {lag_ARIMA_selection(ytw[cols])}')

(127, 28)


In [29]:
def get_best_lag(corp, period, ytw):
    '''
    find optimum lag based on optimum VAR BIC
    '''
    # print(ytw.shape)

    cs = f'CS-{corp}-{period}'
    tb = f'TB-{period}-TY'

    cols = ytw[[cs, tb]] #  ,f'TB-{period}-TY'] # only one column for ARIMA
    aic, bic, hqic = lag_VAR_selection(cols)
    ic_results = []
    ic_results.append([cs, tb, aic, bic, hqic])
    return ic_results

In [30]:

ytw = cslib.get_ytw_from_date(start=start_date, srcfile='YTW-All-Values.xlsx')

print(f'determine best ic. start date: {start_date}')
print(f"{get_best_lag('Aaa', '3MO', ytw)} :")
print(f"{get_best_lag('Aa', '3MO', ytw)} :")
print(f"{get_best_lag('A', '3MO', ytw)} :")
print(f"{get_best_lag('Baa', '3MO', ytw)} :")

print(f"{get_best_lag('Aaa', '1YR', ytw)} :")
print(f"{get_best_lag('Aa', '1YR', ytw)} :")
print(f"{get_best_lag('A', '1YR', ytw)} :")
print(f"{get_best_lag('Baa', '1YR', ytw)} :")

print(f"{get_best_lag('Aaa', '5YR', ytw)} :")
print(f"{get_best_lag('Aa', '5YR', ytw)} :")
print(f"{get_best_lag('A', '5YR', ytw)} :")
print(f"{get_best_lag('Baa', '5YR', ytw)} :")

'''
import pandas as pd
pd.DataFrame(data=)
res = pd.DataFrame(data=ic_results, columns=('cs', 'aic', 'bic', 'hqic'))


print(f'df head: {res.head()}')

period = '5YR' # 3MO, 1YR, 5YR
    corp = 'Aaa'

get_best_lag

print(f'column: {cs} optimal lag - aic: {aic} bic: {bic} hqic: {hqic}')

# add ic results to an array to append to dataframe
'''

determine best ic. start date: 2009-01-31
[['CS-Aaa-3MO', 'TB-3MO-TY', 30, 3, 3]] :
[['CS-Aa-3MO', 'TB-3MO-TY', 29, 1, 4]] :
[['CS-A-3MO', 'TB-3MO-TY', 8, 2, 5]] :
[['CS-Baa-3MO', 'TB-3MO-TY', 8, 3, 4]] :
[['CS-Aaa-1YR', 'TB-1YR-TY', 30, 4, 4]] :
[['CS-Aa-1YR', 'TB-1YR-TY', 4, 4, 4]] :
[['CS-A-1YR', 'TB-1YR-TY', 5, 4, 5]] :
[['CS-Baa-1YR', 'TB-1YR-TY', 5, 4, 4]] :
[['CS-Aaa-5YR', 'TB-5YR-TY', 5, 3, 3]] :
[['CS-Aa-5YR', 'TB-5YR-TY', 30, 3, 5]] :
[['CS-A-5YR', 'TB-5YR-TY', 30, 3, 5]] :
[['CS-Baa-5YR', 'TB-5YR-TY', 30, 3, 5]] :


"\nimport pandas as pd\npd.DataFrame(data=)\nres = pd.DataFrame(data=ic_results, columns=('cs', 'aic', 'bic', 'hqic'))\n\n\nprint(f'df head: {res.head()}')\n\nperiod = '5YR' # 3MO, 1YR, 5YR\n    corp = 'Aaa'\n\nget_best_lag\n\nprint(f'column: {cs} optimal lag - aic: {aic} bic: {bic} hqic: {hqic}')\n\n# add ic results to an array to append to dataframe\n"

### call VAR with maxlags to return t-stat and p-value

In [1]:
'''
call VAR with maxlags to return t-stat and p-value
'''
from statsmodels.tsa.api import VAR
import numpy as np

ytw = cslib.get_ytw_from_date(start=start_date, srcfile='YTW-All-Values.xlsx')
cs, tb, opt_aic = 'CS-Aaa-3MO', 'TB-3MO-TY', 3

endog = ytw[[cs, tb]]
model_fit = VAR(endog=endog).fit(maxlags=opt_aic, trend='ct', verbose=True)
model_fit.summary()

NameError: name 'cslib' is not defined

In [32]:
!pwd

/Users/Justin/Desktop/git-hub-justin-grigg/testPythonDissAnalysis/src
